# FIKH Model: Creep Response

## Protocol-Specific Context

**Creep** (constant stress) reveals **delayed yielding** and **viscosity bifurcation** in FIKH:

1. **Below yield stress**: Initial elastic deformation + slow structural breakdown
2. **Intermediate stress**: Delayed avalanche-like yielding with delay time $t_d \sim \tau_{thix} \left(\frac{\sigma_y(1) - \sigma_0}{\sigma_0}\right)^{1/\alpha}$
3. **Above yield stress**: Immediate plastic flow + potential thermal runaway

**Why this matters**: The $1/\alpha$ exponent means **smaller $\alpha$ gives exponentially longer delays** for the same stress ratio. This is a hallmark of strong memory — the material resists yielding because it "remembers" its structured state.

> **Physical insight**: Fractional memory slows structural breakdown. With $\alpha = 0.5$, a 10% stress deficit below yield can produce 100× longer delay than classical IKH ($\alpha=1$).

> **Handbook:** See [FIKH Creep Protocol](../../docs/source/models/fikh/fikh.rst#creep-step-stress) for delay time scaling and viscosity bifurcation.

## Learning Objectives

1. Predict FIKH creep curves for stresses below/above yield
2. Measure delay time $t_d$ and verify $t_d \sim (\Delta\sigma)^{-1/\alpha}$ scaling
3. Compare $\alpha = 0.5$ vs $\alpha = 0.9$ delay times
4. Observe viscosity bifurcation: delay → sudden yielding
5. Fit creep data to constrain $\alpha$ from transient dynamics

## Prerequisites

- NB01: Flow curve (yield stress parameters)
- NB03: Relaxation (fractional kinetics basics)

**Estimated Time:** 3-5 minutes (fast), 12-18 minutes (full)

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

# Robust path resolution for execution from any directory
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FIKH

_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    compute_fit_quality,
    get_fikh_param_names,
    load_ml_ikh_creep,
    plot_alpha_sweep,
    print_alpha_interpretation,
    print_convergence_summary,
    print_parameter_comparison,
    save_fikh_results,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

# FAST_MODE: controlled by environment variable
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

jax, jnp = safe_import_jax()
verify_float64()

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"FAST_MODE: {FAST_MODE}")

## 2. Theory: Fractional Creep

Creep response under constant applied stress reveals **delayed yielding** and **viscosity bifurcation**.

### Creep Regimes

1. **Elastic response** ($t \ll \tau$): $\gamma \approx \sigma/G$
2. **Delayed yielding**: Strain accelerates as structure breaks down
3. **Steady flow** ($t \gg \tau$): $\dot{\gamma} \to \dot{\gamma}_{ss}$

### Alpha Effect on Creep

- **Lower alpha**: Slower approach to yielding, stretched acceleration
- **Higher alpha**: Sharper yield transition (classical behavior)

## 3. Load Data

In [ ]:
# Load creep data from ML-IKH Excel
STRESS_PAIR = 0  # First stress jump (3 Pa → 5 Pa)
time_data, gamma_dot_data, sigma_init, sigma_final = load_ml_ikh_creep(stress_pair_index=STRESS_PAIR)

print(f"Data points: {len(time_data)}")
print(f"Time range: [{time_data.min():.4f}, {time_data.max():.2f}] s")
print(f"Shear rate range: [{gamma_dot_data.min():.4g}, {gamma_dot_data.max():.4g}] 1/s")
print(f"Stress jump: {sigma_init} Pa → {sigma_final} Pa")

In [ ]:
# Compute strain from shear rate (cumulative integration)
strain_data = np.cumsum(gamma_dot_data[:-1] * np.diff(time_data))
strain_data = np.insert(strain_data, 0, 0.0)

# Plot raw data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogy(time_data, gamma_dot_data, "ko", markersize=5)
ax1.set_xlabel("Time [s]", fontsize=12)
ax1.set_ylabel("Shear rate [1/s]", fontsize=12)
ax1.set_title(f"Creep Response: σ = {sigma_final} Pa", fontsize=13)
ax1.grid(True, alpha=0.3)

ax2.plot(time_data, strain_data, "ko", markersize=5)
ax2.set_xlabel("Time [s]", fontsize=12)
ax2.set_ylabel("Strain [-]", fontsize=12)
ax2.set_title("Cumulative Strain", fontsize=13)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
# Create and fit FIKH model
model = FIKH(include_thermal=False, alpha_structure=0.7)

t0 = time.time()
model.fit(time_data, strain_data, test_mode="creep", sigma_applied=sigma_final, method='scipy')
t_nlsq = time.time() - t0

param_names = get_fikh_param_names(include_thermal=False)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Compute fit quality and plot with uncertainty band
strain_pred = model.predict(time_data, test_mode="creep", sigma_applied=sigma_final)
metrics = compute_fit_quality(strain_data, strain_pred)

print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g}")

fig, ax = plot_nlsq_fit(
    time_data, strain_data, model, test_mode="creep",
    param_names=param_names, log_scale=True,
    xlabel="Time [s]", ylabel="Strain [-]",
    title=f"FIKH Creep Fit (R$^2$ = {metrics['R2']:.5f})",
    sigma_applied=sigma_final,
)
display(fig)
plt.close(fig)

## 5. Alpha Exploration

In [ ]:
# Define fine time grid for alpha sweep
time_fine = np.linspace(time_data.min(), time_data.max(), 200)

# Alpha sweep for creep
alpha_values = [0.3, 0.5, 0.7, 0.9, 0.99]

fig = plot_alpha_sweep(
    model,
    protocol="creep",
    alpha_values=alpha_values,
    x_data=time_fine,
    sigma_applied=sigma_final,
    figsize=(14, 5),
)

fig.axes[0].plot(time_data, strain_data, "ko", markersize=3, alpha=0.5, label="Data")
fig.axes[0].legend(fontsize=8, loc="best")

display(fig)
plt.close(fig)

In [ ]:
# Physical interpretation
fitted_alpha = model.parameters.get_value("alpha_structure")
print_alpha_interpretation(fitted_alpha)

## 6. Bayesian Inference

In [ ]:
# Bayesian inference
initial_values = {name: model.parameters.get_value(name) for name in param_names}

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    time_data,
    strain_data,
    test_mode="creep",
    sigma_applied=sigma_final,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence
all_pass = print_convergence_summary(result, param_names)

print("\n### Bayesian Diagnostic Interpretation")
print("| Metric | Target | Interpretation |")
print("|--------|--------|----------------|")
print("| **R-hat** | < 1.01 | Chain convergence (< 1.05 acceptable) |")
print("| **ESS** | > 400 | Effective sample size (> 100 minimum) |")
print("| **Divergences** | < 1% | NUTS sampling quality |")
print("\nPassing diagnostics ensure reliable parameter estimates and credible intervals.")

### Convergence Diagnostics

**Bayesian Diagnostic Interpretation:**

| Metric | Target | Interpretation |
|--------|--------|----------------|
| **R-hat** | < 1.01 | Chain convergence (< 1.05 acceptable) |
| **ESS** | > 400 | Effective sample size (> 100 minimum) |
| **Divergences** | < 1% | NUTS sampling quality |

In [ ]:
# ArviZ diagnostic plots (trace, pair, forest, energy, autocorr, rank)
display_arviz_diagnostics(result, param_names, fast_mode=FAST_MODE)

In [ ]:
# Parameter comparison
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, param_names)

## 7. Save Results

In [ ]:
save_fikh_results(model, result, "fikh", "creep", param_names)
print("\nResults saved.")

## Key Takeaways

1. **Creep delay time scales as $t_d \sim (\Delta\sigma)^{-1/\alpha}$**
2. **Lower $\alpha$ → exponentially longer delays** for same stress deficit
3. **Viscosity bifurcation**: Delayed yielding followed by avalanche
4. **Fractional memory resists structural breakdown** under constant stress
5. **Creep provides independent constraint on $\alpha$** via transient dynamics
6. **Residual analysis** validates delayed yielding dynamics

---

## Further Reading

- **[FIKH Creep Protocol](../../docs/source/models/fikh/fikh.rst#creep-step-stress)**: Delay time formula and bifurcation analysis
- **[Delayed Yielding](../../docs/source/models/fikh/fikh.rst#viscosity-bifurcation)**: Physical mechanism and industrial implications

### Key References

1. Dimitriou, C.J. & McKinley, G.H. (2014). "A comprehensive constitutive law for waxy crude oil: a thixotropic yield stress fluid." *Soft Matter*, 10, 6619-6644.
2. Geri, M. et al. (2017). "Thermokinematic memory and the thixotropic elasto-viscoplasticity of waxy crude oils." *J. Rheol.*, 61(3), 427-454.

### Next Steps

**Next**: NB05 (SAOS) — Cole-Cole depression and frequency-domain fractional response